In [2]:
!pip install -i https://test.pypi.org/simple my-krml-25176165==2025.0.2.1 --upgrade --extra-index-url https://pypi.org/simple

Looking in indexes: https://test.pypi.org/simple, https://pypi.org/simple
  Attempting uninstall: my-krml-25176165
    Found existing installation: my_krml_25176165 2025.0.2.0
    Uninstalling my_krml_25176165-2025.0.2.0:
      Successfully uninstalled my_krml_25176165-2025.0.2.0

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
from pathlib import Path
import sys
sys.path.insert(0, str(Path.cwd().parents[2] / "src"))

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
!mkdir -p ../../data/raw

!curl -L -o ../../data/raw/insurance.csv \
https://raw.githubusercontent.com/aso-uts/labs_datasets/main/36120-adv_mla/lab02/insurance.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1983k  100 1983k    0     0  88869      0  0:00:22  0:00:22 --:--:--  369k  0     0      0      0 --:--:--  0:00:08 --:--:--     00  0:01:13  0:00:19  0:00:54  103k     0  50474      0  0:00:40  0:00:22  0:00:18  141k


In [15]:
import pandas as pd
import numpy as np
df = pd.read_csv("../../data/raw/insurance.csv")

df.head()       
df.shape        
df.info()       
df.describe(include="all") 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       50000 non-null  int64  
 1   sex       50000 non-null  object 
 2   bmi       50000 non-null  float64
 3   children  50000 non-null  int64  
 4   smoker    50000 non-null  object 
 5   region    50000 non-null  object 
 6   charges   50000 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 2.7+ MB


,age,sex,bmi,children,smoker,region,charges
count,50000.000000,50000,50000.000000,50000.000000,50000,50000,50000.000000
unique,NaN,2,NaN,NaN,2,4,NaN
top,NaN,male,NaN,NaN,no,southeast,NaN
freq,NaN,25176,NaN,NaN,38976,14197,NaN
mean,39.463120,NaN,30.713734,1.113760,NaN,NaN,13343.216363
std,14.117142,NaN,6.092727,1.212835,NaN,NaN,12131.222744
min,18.000000,NaN,17.291000,0.000000,NaN,NaN,1137.535900
25%,27.000000,NaN,26.600000,0.000000,NaN,NaN,4694.431800
50%,40.000000,NaN,30.300000,1.000000,NaN,NaN,9399.232775
75%,51.000000,NaN,34.570000,2.000000,NaN,NaN,17340.746925


In [16]:
df_cleaned = df.copy()
target = df_cleaned.pop("charges")

num_cols = list(df_cleaned.select_dtypes("number").columns)
cat_cols = list(set(df_cleaned.columns) - set(num_cols))
num_cols, cat_cols

from sklearn.preprocessing import StandardScaler, OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore")
features = ohe.fit_transform(df_cleaned[cat_cols])
features = pd.DataFrame(features, columns=ohe.get_feature_names_out())
scaler = StandardScaler()

features[num_cols] = scaler.fit_transform(df_cleaned[num_cols])
features.head()

from joblib import dump
import os
os.makedirs("../../models", exist_ok=True)

dump(ohe, "../../models/ohe.joblib")
dump(scaler, "../../models/scaler.joblib")

['../../models/scaler.joblib']

In [17]:
# Split Dataset

In [18]:
from my_krml_25176165.data.sets import split_sets_random
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(features, target, test_ratio=0.2)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(30000, 8)
(10000, 8)
(10000, 8)


In [19]:
import os
from my_krml_25176165.data.sets import split_sets_random, save_sets

os.makedirs('../data/processed', exist_ok=True)

X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(features, target, test_ratio=0.2)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

save_sets(X_train, y_train, X_val, y_val, X_test, y_test, path='../data/processed/')



(30000, 8)
(10000, 8)
(10000, 8)


In [20]:
from sklearn.dummy import DummyRegressor
from my_krml_25176165.models.performance import print_regressor_scores

# 建立 baseline model
base_reg = DummyRegressor(strategy="mean")

# 訓練 baseline model
base_reg.fit(X_train, y_train)

# 對訓練集做預測
y_base_train = base_reg.predict(X_train)

# 計算 RMSE 和 MAE
print_regressor_scores(y_preds=y_base_train, y_actuals=y_train, set_name="Training")


RMSE Training: 12101.506007041873
MAE Training: 9111.058822092036


In [21]:
# Get Baseline Model

In [22]:
from my_krml_25176165.models.performance import print_regressor_scores
print_regressor_scores(y_preds=y_base_train, y_actuals=y_train, set_name="Training")


RMSE Training: 12101.506007041873
MAE Training: 9111.058822092036
